## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configCopy1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pirovskoye,RU,57.63,92.27,47.32,92,47,8.25,scattered clouds
1,1,Abomsa,ET,9.98,39.98,71.38,64,74,5.32,light rain
2,2,Yellowknife,CA,62.46,-114.35,37.40,37,20,8.05,few clouds
3,3,Vaini,TO,-21.20,-175.20,75.20,78,40,13.87,scattered clouds
4,4,Hithadhoo,MV,-0.60,73.08,81.18,72,91,7.38,overcast clouds


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?75
What is the maximum temperature you would like for your vacation?90


In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Vaini,TO,-21.20,-175.20,75.20,78,40,13.87,scattered clouds
4,4,Hithadhoo,MV,-0.60,73.08,81.18,72,91,7.38,overcast clouds
8,8,Morehead,US,37.27,-87.18,80.60,60,1,4.70,clear sky
11,11,Puerto Ayora,EC,-0.74,-90.35,78.80,57,20,23.04,few clouds
14,14,Alofi,NU,-19.06,-169.92,78.80,65,75,19.46,broken clouds
19,19,Avarua,CK,-21.21,-159.78,78.80,65,76,14.99,broken clouds
20,20,Pemangkat,ID,1.17,108.97,76.37,90,98,6.29,overcast clouds
21,21,Mecca,SA,21.43,39.83,86.47,60,12,4.50,few clouds
23,23,Kapaa,US,22.08,-159.32,86.00,62,75,9.17,broken clouds
28,28,Karratha,AU,-20.74,116.85,76.77,35,10,8.08,clear sky


In [51]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                183
City                   183
Country                183
Lat                    183
Lng                    183
Max Temp               183
Humidity               183
Cloudiness             183
Wind Speed             183
Current Description    183
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Vaini,TO,-21.20,-175.20,75.20,78,40,13.87,scattered clouds
4,4,Hithadhoo,MV,-0.60,73.08,81.18,72,91,7.38,overcast clouds
8,8,Morehead,US,37.27,-87.18,80.60,60,1,4.70,clear sky
11,11,Puerto Ayora,EC,-0.74,-90.35,78.80,57,20,23.04,few clouds
14,14,Alofi,NU,-19.06,-169.92,78.80,65,75,19.46,broken clouds


In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,
4,Hithadhoo,MV,81.18,overcast clouds,-0.60,73.08,
8,Morehead,US,80.60,clear sky,37.27,-87.18,
11,Puerto Ayora,EC,78.80,few clouds,-0.74,-90.35,
14,Alofi,NU,78.80,broken clouds,-19.06,-169.92,
19,Avarua,CK,78.80,broken clouds,-21.21,-159.78,
20,Pemangkat,ID,76.37,overcast clouds,1.17,108.97,
21,Mecca,SA,86.47,few clouds,21.43,39.83,
23,Kapaa,US,86.00,broken clouds,22.08,-159.32,
28,Karratha,AU,76.77,clear sky,-20.74,116.85,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [58]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
4,Hithadhoo,MV,81.18,overcast clouds,-0.60,73.08,Scoop Guest House
8,Morehead,US,80.60,clear sky,37.27,-87.18,CCI Express Inn
11,Puerto Ayora,EC,78.80,few clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
14,Alofi,NU,78.80,broken clouds,-19.06,-169.92,Taloa Heights
19,Avarua,CK,78.80,broken clouds,-21.21,-159.78,Paradise Inn
20,Pemangkat,ID,76.37,overcast clouds,1.17,108.97,Kost H.Syamsuddin
21,Mecca,SA,86.47,few clouds,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
23,Kapaa,US,86.00,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
28,Karratha,AU,76.77,clear sky,-20.74,116.85,ibis Styles Karratha


In [61]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [62]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [67]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))